In [1]:
import os
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

In [2]:
#load data
DATAPATH = '../../datasets'
data = loadmat(os.path.join(DATAPATH, 'ex3data1.mat'))
data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X = data['X']  
y = data['y']  
X.shape, y.shape

((5000, 400), (5000, 1))

In [4]:
# initial setup
input_size = 400  
hidden_size = 25  
num_labels = 10  
lr = 1

In [5]:
from sklearn.preprocessing import OneHotEncoder  
encoder = OneHotEncoder(sparse=False)  
y_onehot = encoder.fit_transform(y)  
y_onehot.shape

(5000, 10)

In [6]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [7]:
def sigmoid_gradient(z):  
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [8]:
params = (np.random.random(size=hidden_size * (input_size+1) + num_labels * (hidden_size+1)) - 0.5) * 0.25
params.shape

(10285,)

In [9]:
m = X.shape[0]  
X = np.matrix(X)  
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
w1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
w2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

w1.shape, w2.shape

((25, 401), (10, 26))

In [10]:
'''
def feed_forward_p(X, w1, w2):
    m = X.shape[0]
    z1 = np.dot(X, w1.T) + np.ones(hidden_size)
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2.T) + np.ones(num_labels)
    a2 = sigmoid(z2)
    return z1, a1, z2, a2
'''
def forward_propagate(X, theta1, theta2):  
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

In [11]:
def cost(X, y, w1, w2, h, learning_rate):  
    
    # compute the cost
    first_term = np.multiply(-y, np.log(h))
    second_term = np.multiply((1 - y), np.log(1 - h))
    J = np.sum(first_term - second_term)

    J = J / m
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(w1[:,1:], 2)) + np.sum(np.power(w2[:,1:], 2)))  

    return J

In [12]:
a1, z2, a2, z3, h = forward_propagate(X, w1, w2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [13]:
cost(X, y_onehot,w1, w2, h, lr)

7.032118844707563

In [14]:
def backprop(X, y, w1, w2, learning_rate):  
    ##### this section is identical to the cost function logic we already saw #####
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, w1, w2)
    cost_err = cost(X, y, w1, w2, h, learning_rate)
    print('i is {} and cost {}'.format(i, cost_err))
    ##### end of cost function logic, below is the new part #####

    delta1 = np.zeros(w1.shape)  # (25, 401)
    delta2 = np.zeros(w2.shape)  # (10, 26)
    d3 = h - y  # (1, 10)
    #print(np.sum(d3))
    z2 = np.insert(z2, 0, values=np.ones(1), axis=1)  # (1, 26)
    d2 = np.multiply((w2.T * d3.T).T, sigmoid_gradient(z2))  # (1, 26)

    delta1 = delta1 + (d2[:,1:]).T * a1
    delta2 = delta2 + d3.T * a2

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (w1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (w2[:,1:] * learning_rate) / m

    w1, w2 = delta1, delta2
    return w1, w2

In [18]:
def backprop1(X, y, z1, a1, z2, a2, w1, w2):
    m = X.shape[0]
    error = a2 - y
    dz2 = np.multiply((w2.T * error.T).T, sigmoid_gradient(z2))
    dw2 = np.dot(dz2, a1.T)/m
    #db2 = np.sum(error, axis=1, keep_dims=True)/m
    da1 = np.dot(, dz2)/m
    dz1 = np.multiply((w1.T * error.T).T, sigmoid_gradient(z1))
    dw1 = np.dot(dz1, X.T)/m
    #db1 = np.sum(dz1, axis=1, keep_dims=True)/m
    return dw2, dw1#, dw1, db1

SyntaxError: invalid syntax (<ipython-input-18-5cf234505695>, line 7)

In [15]:
#w1, w2 = backprop(X, y_onehot,w1, w2,lr)
for i in range(5):
     w1, w2 = backprop(X, y_onehot,w1, w2,lr)

i is 0 and cost 7.032118844707563
i is 1 and cost 26.701236869061493
i is 2 and cost 102.73917270927211
i is 3 and cost 116.53754507631967
i is 4 and cost 117.0875157376301


In [16]:
a1, z2, a2, z3, h = forward_propagate(X, w1, w2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  
y_pred 

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [17]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 10.0%
